In [1]:
import matplotlib
#matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import dlc_bci as bci
from utility import * 

## Get Data
Fetches the data from the server and saves it to a folder at the root 

In [3]:
train_input_100, train_target_100, test_input_100, test_target_100 = import100HzData()
train_input, train_target, test_input, test_target = import1000HzData()

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
print(train_input.shape) 
print(train_target.shape) 
print(test_input.shape) 
print(test_target.shape) 

torch.Size([316, 28, 500])
torch.Size([316])
torch.Size([100, 28, 500])
torch.Size([100])


## Visualize the data

In [5]:
#rawDataForSingleElectrodeVisualization(train_input)

In [6]:
# Data Augmentation 

In [7]:
def preprocessing_train(train_input):
    #denoise and normalize data (without detrending and so)
    tmp = np.array(train_input)
    #tmp = denoisedSignals(tmp)

    #we take the 1000 hz signal and we subsample it with different time steps we downsample to 100Hz but with different initial points. By ding so we have 10 times more samples instead of having ones big weigth for the 1000hz sample

    augemented_train_input = tmp[:,:,0::10]

    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,1::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,2::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,3::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,4::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,5::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,6::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,7::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,8::10]))
    augemented_train_input = np.concatenate((augemented_train_input, tmp[:,:,9::10])) #100 subsampled terms 

    noise_tensor = np.zeros(augemented_train_input.shape)

    for i in range (augemented_train_input.shape[0]):
        noiseIntensity = 0.1*np.max(augemented_train_input[i,:,:])
        noise_tensor[i , :, :] = noise(augemented_train_input[i,:,:], noiseIntensity)
    
    return noise_tensor

def preprocessing_test(test_input):
    #denoise and normalize data (without detrending and so)
    tmp = np.array(test_input)
    tmp = denoisedSignals(tmp)

    #we take the 1000 hz signal and we subsample it with different time steps we downsample to 100Hz but with different initial points. By ding so we have 10 times more samples instead of having ones big weigth for the 1000hz sample
    return tmp

# Neural Net - CNN

In [8]:
# Copy the neural network from the Neural Networks section before and modify it to take 18-channel signal (instead of 1-channel images as it was defined).

2. Define a Convolution Neural Network

In [9]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.n_filters_time=25
        self.n_filters_spat=25
        self.n_filters_conv=25
        self.n_input_channel=28
        self.final_conv_length=2 #!!!!!
        self.filter_time_length=int(10/3)
        self.conv_stride = 1 #Stride after pooling 
        self.pool_stride = 2#3 #Stride after pooling
        self.batch_norm_alpha=0.1
        self.pool_time_length=3
        self.pool_time_stride=3
        self.n_filters_2=50
        self.filter_length_2=int(10/3)
        self.n_filters_3=100
        self.filter_length_3=int(10/3)
        self.n_filters_4=200
        self.filter_length_4=int(10/3)
        self.drop_prob=0.3
        self.batch_norm=True
        self.n_classes=2

        # Layer 1 - REMINDER nn.Conv2d(nbChannels, outputChannels, kxk Square convolution)
        self.a = nn.Conv2d(1, self.n_filters_time, (1, self.filter_time_length), stride = 1) #'Time Convolution' 
        self.b = nn.Conv2d(25, self.n_filters_spat, (self.n_input_channel, 1), stride=(self.conv_stride, 1), bias=False) #'Spatial Convolution' 
        self.c = nn.BatchNorm2d(self.n_filters_conv, momentum=self.batch_norm_alpha, affine=True, eps=1e-5)
        self.d = nn.MaxPool2d(kernel_size=(1, self.pool_time_length), stride=(1, self.pool_stride))

        # Layer 2
        self.e = nn.Dropout(p=self.drop_prob)
        self.f = nn.Conv2d(self.n_filters_conv, self.n_filters_2,(1, self.filter_length_2),stride=(1, self.conv_stride),
                                   bias=False)
        self.g = nn.BatchNorm2d(self.n_filters_2,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.h = nn.MaxPool2d(kernel_size=(1, self.pool_time_length),stride=(1, self.pool_stride))

        # Layer 3
        self.i = nn.Dropout(p=self.drop_prob)
        self.j = nn.Conv2d(self.n_filters_2, self.n_filters_3, (1, self.filter_length_3), stride=(1, self.conv_stride),
                                   bias=False)
        self.k = nn.BatchNorm2d(self.n_filters_3,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.l = nn.MaxPool2d(kernel_size=(1, self.pool_time_length), stride=(1, self.pool_stride))

        # Layer 4
        self.m =nn.Dropout(p=self.drop_prob)
        self.n =nn.Conv2d(self.n_filters_3, self.n_filters_4,
                                   (1, self.filter_length_4),
                                   stride=(1, self.conv_stride),
                                   bias=False)
        self.o =nn.BatchNorm2d(self.n_filters_4,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.p =nn.MaxPool2d(kernel_size=(1, self.pool_time_length),stride=(1, self.pool_stride-1))

        #Classifier Layer
        self.q = nn.Conv2d(self.n_filters_4, self.n_classes, (1, self.final_conv_length), bias=True)
        self.r =nn.LogSoftmax(dim=-1)

        self.classifier = nn.Sequential(
            nn.Linear(100*1*3, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 2),)
        
        
    def forward(self, x):
        print(x.shape)
        x = F.relu(self.a(x))
        print(x.shape)
        x = F.relu(self.b(x))
        print(x.shape)
        x = self.c(x)
        print(x.shape)
        x = self.d(x)
        print(x.shape)
        x = self.e(x)
        print(x.shape)
        x = F.relu(self.f(x))
        print(x.shape)
        x = self.g(x)
        print(x.shape)
        x = self.h(x)
        print('h', x.shape)
        x = self.i(x)
        print('i', x.shape)
        x = F.relu(self.j(x))
        print('j', x.shape)
        x = self.k(x)
        print('k', x.shape)
        x = self.l(x)
        print(x.shape)
        x = self.m(x)
        print(x.shape)
        #x = F.relu(self.n(x))
        #print(x.shape)
        #x = self.o(x)
        #print(x.shape)
        #x = self.p(x)
        #print('p', x.shape)
        x = self.r(x)
        
        
        x = x.view(-1, 100*1*3)
        #print(x.shape)
        x = self.classifier(x)
        print(x.shape)

        return x
net = Net()

3. Define a Loss function and optimizer
Let’s use a Classification Cross-Entropy loss and SGD with momentum.

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#criterion = nn.BCELoss()
#optimizer = optim.Adam(net.parameters())

4. Train the network
This is when things start to get interesting. We simply have to loop over our data iterator, and feed the inputs to the network and optimize.

In [11]:
for epoch in range(1200):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    running_loss = 0.0
   
    usePreprocessing = True
    
    if usePreprocessing : 
        #time = np.linspace(0, 500, 50)
        #normalized_input_train = normalizedSignals(train_input, time, True, 'After Preprocessing - Simple plot of all electrodes - Train')
        preprocessed_input_train = preprocessing_train(train_input)

        inputs = torch.from_numpy(preprocessed_input_train).float()
        inputs = torch.unsqueeze(inputs,1) # make it float and insert a fake batch dimension
        #labels = train_target #y_train
        
        labels_input = train_target #y_train
        labels = labels_input
        for i in range(9): 
            labels = np.concatenate((np.array(labels), np.array(labels_input)), axis = 0)
            
        labels = torch.from_numpy(labels)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.data[0]
        print("Running loss", running_loss)

        # Validation accuracy
        params = ["acc", "auc", "fmeasure"]
        
    else: 
        inputs = train_input #x_train
        inputs = torch.unsqueeze(inputs,1) # make it float and insert a fake batch dimension
        labels = train_target #y_train

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.data[0]
        print("Running loss", running_loss)

        # Validation accuracy
        params = ["acc", "auc", "fmeasure"]


Epoch  0
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.7506510019302368

Epoch  1
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.7428728938102722

Epoch  2
torc

Running loss 0.702426552772522

Epoch  17
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.6988295912742615

Epoch  18
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 

j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.6959630846977234

Epoch  34
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.6940299272537231

Epoch  35
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size

h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.6940654516220093

Epoch  51
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.6959559917449951

Epoch  52
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size

torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.6906386613845825

Epoch  68
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.690365195274353

Epoch  69
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3

...

k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.411949098110199

Epoch  1118
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.4096904397010803

Epoch  1119
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.S

k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.41853785514831543

Epoch  1135
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.41800934076309204

Epoch  1136
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torc

k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.4094475507736206

Epoch  1152
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.40190204977989197

Epoch  1153
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch

k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.39520263671875

Epoch  1169
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.40883809328079224

Epoch  1170
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.S

k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.41943421959877014

Epoch  1186
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torch.Size([3160, 100, 1, 8])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 100, 1, 3])
torch.Size([3160, 2])
Running loss 0.39747750759124756

Epoch  1187
torch.Size([3160, 1, 28, 50])
torch.Size([3160, 25, 28, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 48])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 25, 1, 23])
torch.Size([3160, 50, 1, 21])
torch.Size([3160, 50, 1, 21])
h torch.Size([3160, 50, 1, 10])
i torch.Size([3160, 50, 1, 10])
j torch.Size([3160, 100, 1, 8])
k torc

5. Test the network on the test data
We have trained the network for 2 passes over the training dataset. But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.

# Test Dataset - Prediction 

In [12]:
correct = 0
total = 0

if usePreprocessing: 
    #normalized_input_test = normalizedSignals(test_input, time, True, 'After Preprocessing - Simple plot of all electrodes - Test')   
    #prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
    #prediction = test_input_100 #x_train    
    preprocessed = torch.from_numpy(preprocessing_test(test_input_100)).float()
    prediction_squeezed = torch.unsqueeze(preprocessed,1) # make it float and insert a fake batch dimension
    labels = test_target #y_train

    # wrap them in Variable
    predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)

    # forward + backward + optimize
    #The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
    #that the image is of the particular class. So, let’s get the index of the highest energy:
    outputsPred = net(predictionVar) 

    _, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
    total += labels.size(0)
    correct += (predicted == labels).sum()

    print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))
    
else: 
    #prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
    prediction = test_input_100 #x_train
    prediction_squeezed = torch.unsqueeze(prediction,1) # make it float and insert a fake batch dimension
    labels = test_target #y_train

    # wrap them in Variable
    predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)

    # forward + backward + optimize
    #The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
    #that the image is of the particular class. So, let’s get the index of the highest energy:
    outputsPred = net(predictionVar) 

    _, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
    total += labels.size(0)
    correct += (predicted == labels).sum()

    print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))
    

torch.Size([100, 1, 28, 50])
torch.Size([100, 25, 28, 48])
torch.Size([100, 25, 1, 48])
torch.Size([100, 25, 1, 48])
torch.Size([100, 25, 1, 23])
torch.Size([100, 25, 1, 23])
torch.Size([100, 50, 1, 21])
torch.Size([100, 50, 1, 21])
h torch.Size([100, 50, 1, 10])
i torch.Size([100, 50, 1, 10])
j torch.Size([100, 100, 1, 8])
k torch.Size([100, 100, 1, 8])
torch.Size([100, 100, 1, 3])
torch.Size([100, 100, 1, 3])
torch.Size([100, 2])
Accuracy of the network on the test singals: 58 %


In [13]:
predicted.shape

predicted


 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 0
 1
 0
 0
 1
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 0
 0
 1
 0
 0
 0
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 1
 1
 1
 0
[torch.LongTensor of size 100]

In [14]:
labels

labels.shape

torch.Size([100])